In [1]:
import math
import networkx as nx
import json

In [23]:
def haversine(coord1, coord2):
    R = 6371.0
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [33]:
def construct_graph(city_data, distance_max=100):
    G = nx.Graph()
    communes = {item['commune']: item['coordonnees'] for item in city_data}
    
    for commune, coord in communes.items():
        G.add_node(commune, pos=coord)
    
    for commune, coord in communes.items():
        for autre_commune, autre_coord in communes.items():
            if commune != autre_commune:
                dist = haversine(coord, autre_coord)
                if dist <= distance_max:
                    G.add_edge(commune, autre_commune, weight=dist)
    
    return G

In [34]:
def shortest_path(G, source, destination):
    try:
        path = nx.shortest_path(G, source=source, target=destination, weight='weight')
        distance = nx.shortest_path_length(G, source=source, target=destination, weight='weight')
        return path, distance
    except nx.NetworkXNoPath:
        print(f"Aucun chemin trouvé entre {source} et {destination}.")
        return None, None

In [52]:
import json

with open('assets/data.json', 'r') as file:
    data = json.load(file)
city_data = [{'commune': item['COMMUNE'].lower(), 'coordonnees': item['Geo Shape']['coordinates']} for item in data]

In [53]:
distance_max = 100
G = construct_graph(city_data, distance_max=distance_max)

In [54]:
source = "lille"
destination = "bordeaux"

In [55]:
chemin, distance = shortest_path(G, source, destination)

In [56]:
if chemin and distance:
    print(f"Shortest path from {source} to {destination}  :")
    for i, ville in enumerate(chemin[:-1]):
        print(f"Step {i + 1}: {ville} -> {chemin[i + 1]}")
    print("Total distance:", round(distance, 2), "km")


Shortest path from lille to bordeaux  :
Step 1: lille -> roeux
Step 2: roeux -> gannes
Step 3: gannes -> neuville-sur-oise
Step 4: neuville-sur-oise -> les essarts-le-roi
Step 5: les essarts-le-roi -> villemaury
Step 6: villemaury -> veuzain-sur-loire
Step 7: veuzain-sur-loire -> chambourg-sur-indre
Step 8: chambourg-sur-indre -> jardres
Step 9: jardres -> civray
Step 10: civray -> chateauneuf-sur-charente
Step 11: chateauneuf-sur-charente -> cubzac-les-ponts
Step 12: cubzac-les-ponts -> bordeaux
Total distance: 761.31 km


In [60]:
for node, data in G.nodes(data=True):
    if isinstance(data.get('pos'), list):
        data['pos'] = ','.join(map(str, data['pos']))

nx.write_graphml(G, "graphe_villes.graphml")